4.5 학습 알고리즘 구현
    - 신경망 학습 절차(확률적 경사 하강법, stochastic gradient descent, SGD)
        1. 미니배치: 훈련 데이터 중 일부 추출(이 일부가 미니배치), 미니배치의 손실함수 값을 줄이는 것이 학습 목표
        2. 기울기 산출: 각 가중치 매개변수의 기울기 연산, 이 값으로 손실 함수의 값을 가장 작게 하는 방향 제시
        3. 매개변수 갱신: 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
        4. 1~3단계 반복

4.5.1 2층 신경망 클래스 구현
    - 변수
        1. params: 신경망의 매개변수를 보관하는 딕셔너리 변수, n층의 가중치 및 편향
        2. grads: 기울기를 보관하는 딕셔너리 변수, n층의 가중치 및 편향의 기울기
    - 클래스의 메서드
        1. __init__: 초기화 수행, 인수(입력층의 뉴련 수, 은닉층의 뉴런 수, 출력층의 뉴런 수), 클래스 생성 시 불리는 메서드
        2. predict: 예측(추론) 수행
        3. loss: 손실함수 값 연산, 인수(x: 이미지 데이터, t: 정답 레이블)
        4. accuracy: 정확도 연산
        5. numerical_gradient: 가중치 매개변수의 기울기 연산
        6. gradient: 가중치 매개변수의 기울기 연산, 5의 성능 개선

In [20]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

import numpy as np

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화: 정규분포를 따르는 난수
        # 편향: 0으로 초기화
        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(input_size, hidden_size,)
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b2']=np.zeros(output_size)

    def predict(self, x):
        W1, W2=self.params['W1'], self.params['W2']
        b1, b2=self.parmas['b1'], self.params['b2']

        a1=np.dot(x, W1)+b1
        z1=sigmoid(a1)
        a2=np.dot(z1, W2)+b2
        y=softmax(a2)

        return y

    # x: 입력 데이터, t: 정답 레이블-> 입력과 정답으로 교차 엔트로피 오차 연산
    def loss(self, x, t):
        y=self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y=self.predict(x)
        y=np.argmax(y, axis=1)
        t=np.argmax(t, axis=1)

        accuracy=np.sum(y==t)/float(x.shape[0])
        return accuracy

    # 각 매개변수의 기울기 연산, 수치 미분 방식
    def numerical_gradient(self, x, t):
        loss_W=lambda W: self.loss(x, t)

        grads={}
        grads['W1']=numerical_gradient(loss_W, self.params['W1'])
        grads['b1']=numerical_gradient(loss_W, self.params['b1'])
        grads['W2']=numerical_gradient(loss_W, self.params['W2'])
        grads['b2']=numerical_gradient(loss_W, self.params['b2'])

        return grads


In [21]:
net=TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
# 입력 이미지 28*28, 출력 10개, 은닉층은 적당한 사이즈로 설정
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


4.5.2 미니배치 학습 구현
    - 훈련 데이터 중 일부를 무작위로 꺼낸 후(미니배치), 경사법으로 매개변수 연산

In [22]:
import numpy as np
from mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test)=load_mnist(normalize=True, one_hot_label=True)

train_loss_list=[]

# 하이퍼파라미터
liters_num=10000    # 반복 횟수
train_size=x_train.shape[0]
batch_size=100      # 미니배치 크기
learning_rate=0.1
network=TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(liters_num):
    # 미니배치 획득
    batch_mask=np.random.choice(train_size, batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]

    # 기울기 계산 - 경사 하강법 수행
    grad=network.numerical_gradient(x_batch, t_batch)
    # grad = network.graident(x_batch, t_batch) # 성능 개선판

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key]-=learning_rate*grad[key]

    # 학습 경과 기록
    loss=network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

4.5.3 시험 데이터로 평가
    - 범용 능력 평가: 훈련 데이터에 포함되지 않은 데이터 사용해 평가
    - epoch: 학습에서 훈련 데이터를 모두 소진했을 대의 횟수(미니배치)

In [ ]:
import numpy as np
from mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test)=load_mnist(normalize=True, one_hot_label=True)

train_loss_list=[]
train_acc_list=[]
test_acc_list=[]
# 1 epoch당 반복 수
iter_per_epoch=max(train_size/batch_size, 1)

# 하이퍼파라미터
liters_num=10000    # 반복 횟수
train_size=x_train.shape[0]
batch_size=100      # 미니배치 크기
learning_rate=0.1
network=TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(liters_num):
    # 미니배치 획득
    batch_mask=np.random.choice(train_size, batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]

    # 기울기 계산 - 경사 하강법 수행
    grad=network.numerical_gradient(x_batch, t_batch)
    # grad = network.graident(x_batch, t_batch) # 성능 개선판

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key]-=learning_rate*grad[key]

    # 학습 경과 기록
    loss=network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1epoch 당 정확도 계산
    if i%iter_per_epoch==0:
        train_acc=network.accuracy(x_train, t_train)
        test_acc=network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc |"+str(train_acc)+", "+str(test_acc))

- 훈련 데이터와 시험 데이터를 사용하고 평가한 정확도가 차이가 별로 없다==오버피팅 X
- 오버피팅 예방법 중 early stopping!

4.6 정리
    - 신경망 학습의 지표: 손실 함수
    - 손실함수 기준으로 값이 가장 작아지는 가중치 매개변수 도출
    - 수치 미분: 아주 작은 값을 주었을 때의 차분으로 미분 연산, 가중치 매개변수의 기울기 도출 가능, 시간 소요가 좀 있음-> 이는 오차역전파법으로 해결 가능